# Kenga AI - Python Version

Этот ноутбук позволяет запустить Python-версию Kenga AI непосредственно в Google Colab.

## Установка и настройка

In [ ]:
!pip install torch transformers fastapi uvicorn python-multipart plotly pandas scikit-learn opencv-python-headless pytesseract
!apt-get install -y tesseract-ocr

# Клонируем репозиторий
!git clone https://github.com/your-repo/kengapy.git
%cd kengapy/python_version

## Запуск модели

In [ ]:
import sys
sys.path.append('.')

from src.neural.models.kenga import KengaModel, KengaConfig
from src.neural.trainers.autonomous_trainer import AutonomousTrainer
from src.monitoring.visualization import MetricsVisualizer

# Инициализация модели
config = KengaConfig(
    vocab_size=50000,
    hidden_size=768,
    num_layers=6,
    num_heads=12
)
model = KengaModel(config)
trainer = AutonomousTrainer(model)
visualizer = MetricsVisualizer()

## Интерактивный интерфейс

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

text_input = widgets.Textarea(
    value='',
    placeholder='Введите ваш запрос здесь...',
    description='Запрос:',
    disabled=False,
    layout={'width': '100%', 'height': '100px'}
)

output = widgets.Output()

def on_send_click(b):
    with output:
        output.clear_output()
        print("Обработка запроса...")
        response = model.process(text_input.value)
        print(f"Ответ: {response}")

send_button = widgets.Button(description='Отправить')
send_button.on_click(on_send_click)

file_upload = widgets.FileUpload(
    accept='',  # Принимаем все типы файлов
    multiple=True,
    description='Загрузить файлы'
)

display(text_input, send_button, file_upload, output)

## Визуализация метрик

In [ ]:
import plotly.graph_objects as go
from IPython.display import display

def plot_metrics():
    metrics = trainer.get_metrics()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=metrics['loss'], name='Loss'))
    fig.add_trace(go.Scatter(y=metrics['accuracy'], name='Accuracy'))
    
    fig.update_layout(
        title='Training Metrics',
        xaxis_title='Step',
        yaxis_title='Value'
    )
    
    fig.show()

# Кнопка для обновления графиков
update_button = widgets.Button(description='Обновить метрики')
update_button.on_click(lambda b: plot_metrics())
display(update_button)

## Анализ файлов

In [ ]:
from src.file_analysis.analyzer import FileAnalyzer

analyzer = FileAnalyzer()

def analyze_uploaded_files(change):
    with output:
        output.clear_output()
        for filename, file_info in change['new'].items():
            content = file_info['content']
            with open(filename, 'wb') as f:
                f.write(content)
            
            print(f"Анализ файла {filename}...")
            results = analyzer.analyze_file(filename)
            print(f"Результаты:\n{results}\n")

file_upload.observe(analyze_uploaded_files, names='value')

## Эксперименты

In [ ]:
from src.experiments.manager import ExperimentManager, ExperimentConfig

experiment_manager = ExperimentManager()

def create_experiment():
    config = ExperimentConfig(
        name="Test Experiment",
        description="Test experiment with default parameters",
        model_config=config.__dict__,
        training_config={"batch_size": 32, "learning_rate": 1e-4},
        tags=["test", "default"]
    )
    
    experiment = experiment_manager.create_experiment(config)
    print(f"Created experiment with ID: {experiment.id}")
    return experiment

create_exp_button = widgets.Button(description='Создать эксперимент')
create_exp_button.on_click(lambda b: create_experiment())
display(create_exp_button)